In [81]:
#Importando as bibliotecas
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests_html import HTML
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import sqlite3
import datetime
import os

In [82]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [83]:
#Criando listas
Urls_amazon = []

Urls_amazon_more = []

Products_Links = []

Amazon_price = []
Amazon_price_2 = []

Amazon_seller = []
Amazon_seller_2 = []

Amazon_title = []
Amazon_title_2 = []

Amazon_installment_price_full = []
Amazon_installment_price_full_2 = []


internacional_list = []
more_offers_list = []


#Listas Button Sellers
Amazon_Button_Sellers = []
Amazon_Button_Price = []
Amazon_Button_ASIN = []

#Listas More Sellers
Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
Amazon_ID_more = []

In [84]:
#Listas Button Sellers
Amazon_Button_Sellers = []
Amazon_Button_Price = []
Amazon_Button_ASIN = []

#Listas More Sellers
Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
Amazon_ID_More = []

In [85]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_36032\3380238038.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [86]:
#Função para criar os links de busca
def getting_n_creating_amazon(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "+")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.amazon.com.br/s?k=" + df['Urls']

    return df

In [87]:
Df_Products_raw = getting_n_creating_amazon("Motorola")

In [88]:
#Função pegando Links
def search_links(url):
    global Urls_amazon

    time.sleep(5)

    driver.get(url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)

    Links = [x for x in html_obj.links]
    products_links = [f'https://www.amazon.com.br{x}' for x in Links]

    for link in products_links:
        Urls_amazon.append(link)

    Urls_amazon = [s for s in Urls_amazon if '/dp/' in s]
    Urls_amazon = [s for s in Urls_amazon if not '#customerReviews' in s]

    try:
        Soup = BeautifulSoup(driver.page_source, 'html.parser')
        next_page = "https://www.amazon.com.br" + Soup.find(class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')['href']
        print(next_page)

        search_links(next_page)
    except:
        pass

In [89]:
for url in tqdm(Df_Products_raw['Urls_search']):
    search_links(url)

100%|██████████| 14/14 [02:35<00:00, 11.13s/it]


In [90]:
print(len(Urls_amazon))

290


In [91]:
#Limpando as urls
def clean_link(urls, brand):
    #Criando a conexão com a tabela de exclusão 
    Database_exclusion = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_exclusao"

    #Criando a Query
    query = "SELECT * FROM " + table

    connection = sqlite3.connect(Database_exclusion)

    #Criando o dataset em brando
    df_itens = pd.read_sql_query(query, connection)

    clean_urls = pd.DataFrame()

    clean_urls['Urls_Completas'] = urls
    clean_urls['Urls_limpas'] = clean_urls['Urls_Completas'].str.partition("ref")[0]

    Urls_limpas = clean_urls['Urls_limpas'].tolist()

    for word in df_itens['Words']:
         Urls_limpas = [s for s in Urls_limpas if not word in s]

    clean_urls = pd.DataFrame()

    clean_urls['Urls_finais'] = Urls_limpas

    clean_urls['ASIN'] = clean_urls['Urls_finais'].str.partition("/dp/")[2].str.partition("/")[0]

    clean_urls.drop_duplicates(subset='ASIN',inplace=True)
    clean_urls.reset_index(inplace=True, drop=True)
    return clean_urls

In [92]:
New_urls = clean_link(Urls_amazon, 'Motorola')

In [93]:
New_urls

,Urls_finais,ASIN
0,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,B07BBTDDNF
1,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,B07M9ZDJ7J
2,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,B01IRIVFRS
3,https://www.amazon.com.br/Eletronica-Motorola-...,B084QZ7JPF
4,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,B08QFCNPXJ
5,https://www.amazon.com.br/Motorola-Bab%C3%A1-E...,B09KMFZN2Y
6,https://www.amazon.com.br/eletr%C3%B4nica-term...,B088GQ387H
7,https://www.amazon.com.br/atualizado-infraverm...,B097NDPXBH
8,https://www.amazon.com.br/Acess%C3%B3rio-Eletr...,B09CDGLXR5
9,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,B01N807E5P


In [94]:
def search_atributes(url):
    #Tempo para não haver o bloqueio
    time.sleep(5)

    #Entrando dentro do site com o driver
    driver.get(url)

    #Tempo para carregar
    time.sleep(10)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    #Criando o Soup
    soup = BeautifulSoup(html_str, 'html.parser')

    #Fazendo o try do nome do vendedor
    try:
        seller = soup.find(id='sellerProfileTriggerId').text
        seller_2 = soup.find("a", attrs={"id":'sellerProfileTriggerId'}).text
        Amazon_seller.append(seller)
        Amazon_seller_2.append(seller_2)
    except:
        Amazon_seller.append("Erro")
        Amazon_seller_2.append("Erro")

    #Fazendo o try do preço do produto a vista
    try:
        Div_Price = soup.find('div', attrs={"data-feature-name":"corePrice"})
        price_2 = Div_Price.find(class_='a-offscreen').text
        Amazon_price_2.append(price_2)
    except:
        try:
            Amazon_price_2.append(soup.find(class_='a-button-text')['href'])
        except:
            Amazon_price_2.append("Erro")

    #Pegando o título do produto
    try:
        title = soup.find(id='productTitle').text
        Amazon_title.append(title)
    except:
        Amazon_title.append('Erro')

    #Pegando o internacional
    try:
        soup.find('img', attrs={'data-a-hires':'https://images-na.ssl-images-amazon.com/images/G/32/foreignseller/Foreign_Seller_Badge_v2._CB403622375_.png'})
        internacional_list.append("Internacional")
    except:
        internacional_list.append("Nacional")

    #Fazendo o try para pegar o preço da parcela
    try:
        installment = soup.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full.append(installment)
        
        Div_installment = soup.find('div', attrs={'id':'installmentCalculator_feature_div'})
        Installment_2 = Div_installment.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full_2.append(Installment_2)
    except:
        Amazon_installment_price_full.append("0")
        Amazon_installment_price_full_2.append("Erro")

    #Fazendo o try para ver se tem mais ofertas
    try:
        Main_Div_More_offers = soup.find('div', attrs={"id":"olpLinkWidget_feature_div"})
        Div_More_offers = Main_Div_More_offers.find('div', attrs={'class':'a-section olp-link-widget'})
        Div_More_offers_text = Div_More_offers.find('div', attrs={'class':'olp-text-box'}).text
        more_offers_list.append(Div_More_offers_text)
    except:
        more_offers_list.append("Comparar outras 0 ofertas")


In [95]:
for url in tqdm(New_urls['Urls_finais']):
    search_atributes(url)

100%|██████████| 34/34 [09:48<00:00, 17.32s/it]


In [96]:
print(len(Amazon_title))
print(len(Amazon_price_2))
print(len(Amazon_seller_2))
print(len(more_offers_list))
print(len(Amazon_installment_price_full_2))
print(len(internacional_list))

34
34
34
34
34
34


In [97]:
def dataset_amazon(url, sellers, preco, titulo, more_url):
    # Criando o DataFrame
    Dataset_amazon = pd.DataFrame()

    #
    Hoje = pd.to_datetime('today', errors='ignore').date()

    # Colocando os dados
    Dataset_amazon['URL'] = url
    
    Dataset_amazon['DATE'] = Hoje

    Dataset_amazon['MARKETPLACE'] = "AMAZON"

    #Arrumando a coluna de sellers
    Dataset_amazon['SELLER'] = sellers
    Dataset_amazon['SELLER'] = Dataset_amazon['SELLER'].str.replace("Erro","Amazon", regex=False)

    #Arrumando o preço 
    Dataset_amazon['PRICE'] = preco
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(".","", regex=True)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace("R$","", regex=False)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(",",".", regex=True)

    #Arrumando os valores de installment 
    Dataset_amazon['INSTALLMENT FULL'] = Amazon_installment_price_full
    Dataset_amazon['PARCEL'] = Dataset_amazon['INSTALLMENT FULL'].str.extract('(\d+)')
    Dataset_amazon['PARCEL'] = Dataset_amazon['PARCEL'].astype("int")
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0]
    Dataset_amazon['Installment3'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0].str.partition(",")[2]
    Dataset_amazon['Installment4'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['Installment3'] + "." + Dataset_amazon['Installment4']
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].astype("float")
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].fillna(0)
    Dataset_amazon['INSTALLMENT_PAYMENT'] = Dataset_amazon['PARCEL'] * Dataset_amazon['INSTALLMENT']


    Dataset_amazon['ID'] = Dataset_amazon['URL'].str.partition('/dp/')[2].str.partition('/')[0]
    Dataset_amazon['PRODUCT'] = titulo
    Dataset_amazon['INTERNACIONAL'] = internacional_list

    #Arrumando valores de mais sellers
    Dataset_amazon['MORE'] = more_url
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].str.partition("outras ")[2].str.partition(" ofertas")[0]
    
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].astype('int')
    

    Dataset_amazon = Dataset_amazon.drop(columns=["INSTALLMENT FULL", "Installment3", "Installment4", "parcel_price_bruto"])

    #Pegando os itens certos 
    #Dataset_amazon = Dataset_amazon[Dataset_amazon["PRICE"] != "Erro"]
    #Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].astype('float')


    #Colocando na ordem correta
    Dataset_amazon = Dataset_amazon[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT','INTERNACIONAL', 'MORE']]

    # if brand == "GoPro":
    #     Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 900]
    # elif brand == 'Motorola':
    #     Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 70]
    # elif brand == 'Wacom':
    #     Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 100]
    

    return Dataset_amazon

In [98]:
Df_final = dataset_amazon(New_urls['Urls_finais'], Amazon_seller_2, Amazon_price_2, Amazon_title, more_offers_list)

In [99]:
Df_final=  Df_final.reset_index(drop=True)

In [100]:
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-06-02,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,AMAZON,Amazon,899.00,10,89.90,899.00,B07BBTDDNF,Babá Eletrônica Digital com Câmera MBP...,Internacional,0
1,2022-06-02,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,Amazon,999.00,10,99.90,999.00,B07M9ZDJ7J,Babá Eletrônica Motorola MBP481 - Tela...,Internacional,2
2,2022-06-02,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,Amazon,1592.91,10,159.30,1593.00,B01IRIVFRS,"Babá Eletrônica Motorola MBP36XL, Moto...",Internacional,0
3,2022-06-02,https://www.amazon.com.br/Eletronica-Motorola-...,AMAZON,Thaun Shop,1677.15,10,167.76,1677.60,B084QZ7JPF,Babá Eletronica Motorola MBP855 Connec...,Internacional,3
4,2022-06-02,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,AMAZON,Amazon,1502.91,10,150.30,1503.00,B08QFCNPXJ,"Babá Eletrônica Motorola MBP30A, Motor...",Internacional,2
5,2022-06-02,https://www.amazon.com.br/Motorola-Bab%C3%A1-E...,AMAZON,Amazon,331.25,10,33.17,331.70,B09KMFZN2Y,Babá Eletrônica Motorola AM21,Internacional,0
6,2022-06-02,https://www.amazon.com.br/eletr%C3%B4nica-term...,AMAZON,Wellao,375.00,10,37.50,375.00,B088GQ387H,Babá eletrônica com termômetro baby mo...,Internacional,3
7,2022-06-02,https://www.amazon.com.br/atualizado-infraverm...,AMAZON,♑GMTDDP,352.99,10,35.38,353.80,B097NDPXBH,Monitor de bebê atualizado,Internacional,12
8,2022-06-02,https://www.amazon.com.br/Acess%C3%B3rio-Eletr...,AMAZON,Amazon,169.00,5,33.80,169.00,B09CDGLXR5,"Acessório Babá Eletrônica STARGRIP, Mo...",Internacional,0
9,2022-06-02,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,Amazon,/gp/offer-listing/B01N807E5P/ref=dp_olp_unknow...,0,0.00,0.00,B01N807E5P,Baba Eletrônica Motorola Mbp-855 Conne...,Internacional,0


In [57]:
#Criar a condicional da coluna More
Df_button_sellers = Df_final[Df_final['PRICE'].str.len() > 10]
Df_button_sellers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-06-02,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,Amazon,/gp/offer-listing/B01N807E5P/ref=dp_olp_unknow...,0,0.0,0.0,B01N807E5P,Baba Eletrônica Motorola Mbp-855 Conne...,Internacional,0
20,2022-06-02,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,AMAZON,Amazon,/gp/offer-listing/B078KHB3YW/ref=dp_olp_unknow...,0,0.0,0.0,B078KHB3YW,Babá Eletrônica MBP668Connect com Tela...,Internacional,0
31,2022-06-02,https://www.amazon.com.br/Motorola-Baby-Monito...,AMAZON,Amazon,/gp/offer-listing/B09BP6TMMF/ref=dp_olp_unknow...,0,0.0,0.0,B09BP6TMMF,Motorola Baby Monitor VM855 - monitor ...,Internacional,0
43,2022-06-02,https://www.amazon.com.br/Motorola-Baby-Monito...,AMAZON,Amazon,/gp/offer-listing/B09D1DLK1D/ref=dp_olp_unknow...,0,0.0,0.0,B09D1DLK1D,Motorola Baby Monitor VM85 - monitor d...,Internacional,0


In [58]:
def Get_Button_Sellers(ASIN):
    global Amazon_Button_Sellers

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_Button_Sellers.append(seller.text)


    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'avaliações' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not ' Amazon.com.br ' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'avaliação' in s]
    Amazon_Button_Sellers =  [s for s in Amazon_Button_Sellers if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_Button_Price.append(price.text)
        Amazon_Button_ASIN.append(ASIN)

In [59]:
for item in tqdm(Df_button_sellers['ID']):
    Get_Button_Sellers(item)

100%|██████████| 4/4 [00:41<00:00, 10.29s/it]


In [60]:
print(len(Amazon_Button_Sellers))
print(len(Amazon_Button_Price))
print(len(Amazon_Button_ASIN))

2
2
2


In [61]:
Dataframe_Button = pd.DataFrame()
Dataframe_Button['ID'] = Amazon_Button_ASIN

Dataframe_Button['DATE'] = pd.to_datetime('today', errors='ignore').date()

Dataframe_Button['MARKETPLACE'] = 'AMAZON'

Dataframe_Button['SELLER'] = Amazon_Button_Sellers

Dataframe_Button['PRICE'] = Amazon_Button_Price
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace(".","", regex=True)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace("R$","", regex=False)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].str.replace(",",".", regex=True)
Dataframe_Button['PRICE'] = Dataframe_Button['PRICE'].astype('float')

Dataframe_Button['PARCEL'] = 10

Dataframe_Button['INSTALLMENT'] = Dataframe_Button['PRICE'] / Dataframe_Button['PARCEL']

Dataframe_Button['INSTALLMENT_PAYMENT'] = Dataframe_Button['PRICE'] * Dataframe_Button['PARCEL']

Dataframe_Button['INTERNACIONAL'] = 'ERRO'

url_names = []
for id in Dataframe_Button['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'].values[0])

products_names = []
for id in Dataframe_Button['ID']:
    products_names.append(Df_final.loc[Df_final['ID'] == id,'PRODUCT'].values[0])

Dataframe_Button['URL'] = url_names
Dataframe_Button['PRODUCT'] = products_names

Dataframe_Button

,ID,DATE,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,INTERNACIONAL,URL,PRODUCT
0,B09D1DLK1D,2022-06-02,AMAZON,Triplenet Pricing INC,2911.25,10,291.125,29112.5,ERRO,https://www.amazon.com.br/Motorola-Baby-Monito...,Motorola Baby Monitor VM85 - monitor d...
1,B09BP6TMMF,2022-06-02,AMAZON,Triplenet Pricing INC,2519.10,10,251.910,25191.0,ERRO,https://www.amazon.com.br/Motorola-Baby-Monito...,Motorola Baby Monitor VM855 - monitor ...


In [26]:
Df_more_offers = Df_final[Df_final['MORE'] != 0]
Df_more_offers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-06-01,https://www.amazon.com.br/Display-Interativo-C...,AMAZON,Amazon,6414.31,10,641.44,6414.40,B07L77GTTY,"Display Interativo Cintiq 16, Wacom, D...",Internacional,4
1,2022-06-01,https://www.amazon.com.br/Folha-Textura-Padr%C...,AMAZON,Amazon,127.72,4,31.93,127.72,B07195NDY9,Folha De Textura Wacom G - Ack122312. ...,Internacional,2
2,2022-06-01,https://www.amazon.com.br/Mesa-Digitalizadora-...,AMAZON,INPOWER,199.00,6,33.20,199.20,B0785S55QQ,Wacom One CTL472 - Mesa Digitalizadora...,Internacional,42
3,2022-06-01,https://www.amazon.com.br/Mesa-Digitalizadora-...,AMAZON,Megaprice Comercio,335.40,10,33.54,335.40,B079HL9YSF,Wacom CTL4100 - Mesa Digitalizadora In...,Internacional,33
4,2022-06-01,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,W-TECH IMPORTS,5997.00,10,599.70,5997.00,B01MSXN9M5,Mesa Digitalizadora Intuos Pro Paper G...,Internacional,5
7,2022-06-01,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,3299.99,10,330.08,3300.80,B01MQU5LW7,Mesa Digitalizadora Wacom Intuos Pro M...,Internacional,4
8,2022-06-01,https://www.amazon.com.br/Folha-Textura-%C3%81...,AMAZON,Amazon,299.00,9,33.24,299.16,B071DWY2WP,Folha Textura Wacom Áspera G - Ack1223...,Internacional,3
9,2022-06-01,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,1386.95,1,1.90,1.90,B079NWPZTG,Mesa Digitalizadora Wacom Intuos Pen &...,Internacional,11
10,2022-06-01,https://www.amazon.com.br/Pontas-reposi-caneta...,AMAZON,Amazon,53.00,0,0.00,0.00,B005KB8H7M,Wacom Pontas de reposi o canetas ponte...,Internacional,3
11,2022-06-01,https://www.amazon.com.br/Caneta-Wacom-Tablets...,AMAZON,Amazon,579.90,644,644.33,414948.52,B079J2FSQ7,Mesa Digitalizadora Wacom Bluetooth Pe...,Internacional,16


In [27]:
def search_more_offers_1(ASIN):
    global Amazon_seller_more
    
    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)
    
    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base')[4:]:
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen')[2:]:
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [28]:
def search_more_offers_2(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=2&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [29]:
def search_more_offers_3(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=3&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [30]:
def search_more_offers_4(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=4&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [31]:
def search_more_offers_5(ASIN):
    global Amazon_seller_more

    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=5&experienceId=aodAjaxMain"

    driver.get(new_url)

    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-color-base'):
        Amazon_seller_more.append(seller.text)


    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

    for price in Soup.find_all(class_='a-offscreen'):
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [32]:
for id,more in zip(Df_more_offers.ID, Df_more_offers.MORE):
    if more < 10:
        search_more_offers_1(id)
    elif (more > 10) and (more < 20):
        search_more_offers_1(id)
        search_more_offers_2(id)
    elif (more > 20) and (more < 30):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
    elif (more > 30) and (more < 40):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
        search_more_offers_4(id)
    elif (more > 40) and (more < 50):
        search_more_offers_1(id)
        search_more_offers_2(id)
        search_more_offers_3(id)
        search_more_offers_4(id)
        search_more_offers_5(id)

In [29]:
Amazon_seller_more = []
Amazon_price_more = []
Amazon_ID_More = []

In [33]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

218
218
218


In [95]:
new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B079NWPZTF" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-color-base')[4:]:
    Amazon_seller_more.append(seller.text)


Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliações' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not ' Amazon.com.br ' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'avaliação' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Recém-lançado' in s]

for price in Soup.find_all(class_='a-offscreen')[2:]:
    Amazon_price_more.append(price.text)
    Amazon_ID_More.append('B079NWPZTF')

In [97]:
for name in Amazon_seller_more:
    print(name)

Triplenet Pricing INC
E-SPOT
DONA JUREMA BOUTIK
INPOWER
Amazon.com.br
Megaprice Comercio
MAGIC WORLD ELETRONICOS
REX INFORMATICA
Tudo à Beça
PHOENIX ELETRONICOS
Gigantec
Angel Eletronicos
SHOPAR INFO
Qualitech Informática
Primetek Computadores
Pichau
OFICINA DOS BITS
W-TECH IMPORTS
HD Store Oficial
Loja iPlace
DOTCOMP
BRDF Informática
Triplenet Pricing INC
Infinity Informática
bringIT
E-SPOT
Kadri Tecnologia
RPW SERVIÇOS INTEGRADOS LTDA
2K Shop
StoreAtual. com
Bits & Bytes Tecnologia
Skill Gaming
Unimporte
IBYTE
Tendinfo
INFOSHOP.
Tudo à Beça
OFICINA DOS BITS
Gigantec
Triplenet Pricing INC
W-TECH IMPORTS
TechShed
E-SPOT
Bits & Bytes Tecnologia
Tendinfo
INFOSHOP.
Mil Coisas droship
 Recém-lançado 
Mil Coisas Drop
 Recém-lançado 
Amazon.com.br
Loja iPlace
BalbinoShop
E-SPOT
W-TECH IMPORTS
Tendinfo


In [47]:
for price in Amazon_price_more:
    print(price)

R$ 6.323,80
R$ 8.599,00
R$ 11.499,00


In [98]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

56
54
54


In [78]:
Amazon_seller_more = []
Amazon_price_more = []

In [31]:
Amazon_seller_more

[' INFOSHOP.',
 ' INFOSHOP.',
 ' INFOSHOP.',
 ' INFOSHOP.',
 ' Tudo à Beça',
 ' OFICINA DOS BITS',
 ' Gigantec',
 ' Triplenet Pricing INC',
 ' W-TECH IMPORTS',
 ' TechShed',
 ' E-SPOT',
 ' Bits & Bytes Tecnologia',
 ' Tendinfo']

In [107]:
Amazon_seller_more.append("Erro")

In [34]:
Dataframe = pd.DataFrame()

Dataframe['ID'] = Amazon_ID_More
Dataframe['SELLER'] = Amazon_seller_more
Dataframe['PRICE'] = Amazon_price_more

In [35]:
Dataframe

,ID,SELLER,PRICE
0,B07L77GTTY,Triplenet Pricing INC,"R$ 6.382,25"
1,B07L77GTTY,E-SPOT,"R$ 8.599,00"
2,B07L77GTTY,DONA JUREMA BOUTIK,"R$ 11.499,00"
3,B07195NDY9,Triplenet Pricing INC,"R$ 458,99"
4,B0785S55QQ,BIG BOSS ELETRONICOS,"R$ 204,90"
...,...,...,...
213,B07YFG742J,DONA JUREMA BOUTIK,"R$ 2.135,00"
214,B07YFG742J,Promotion.Shop 💎,"R$ 2.185,45"
215,B084DVTSC9,Triplenet Pricing INC,"R$ 402,14"
216,B084DVTSC9,BUY SUPPLY,"R$ 811,01"


In [32]:
Amazon_seller_more

[' Bebê Veste Luxo',
 ' BabyeKids',
 ' TECNOTOY COMERCIAL',
 ' Avile Shop',
 ' TRP STORE',
 ' VIP STORE BRINQUEDOS']

In [25]:
Amazon_price_more

['R$\xa0349,00']

In [36]:
Dataframe_More = pd.DataFrame()
Dataframe_More['ID'] = Amazon_ID_More

Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()

Dataframe_More['MARKETPLACE'] = 'AMAZON'

Dataframe_More['SELLER'] = Amazon_seller_more

Dataframe_More['PRICE'] = Amazon_price_more
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(".","", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace("R$","", regex=False)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(",",".", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].astype('float')

Dataframe_More['PARCEL'] = 10

Dataframe_More['INSTALLMENT'] = Dataframe_More['PRICE'] / Dataframe_More['PARCEL']

Dataframe_More['INSTALLMENT_PAYMENT'] = Dataframe_More['PRICE'] * Dataframe_More['PARCEL']

Dataframe_More['INTERNACIONAL'] = 'ERRO'

url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'].values[0])

products_names = []
for id in Dataframe_More['ID']:
    products_names.append(Df_final.loc[Df_final['ID'] == id,'PRODUCT'].values[0])

Dataframe_More['URL'] = url_names
Dataframe_More['PRODUCT'] = products_names

Dataframe_More

,ID,DATE,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,INTERNACIONAL,URL,PRODUCT
0,B07L77GTTY,2022-06-01,AMAZON,Triplenet Pricing INC,6382.25,10,638.225,63822.5,ERRO,https://www.amazon.com.br/Display-Interativo-C...,"Display Interativo Cintiq 16, Wacom, D..."
1,B07L77GTTY,2022-06-01,AMAZON,E-SPOT,8599.00,10,859.900,85990.0,ERRO,https://www.amazon.com.br/Display-Interativo-C...,"Display Interativo Cintiq 16, Wacom, D..."
2,B07L77GTTY,2022-06-01,AMAZON,DONA JUREMA BOUTIK,11499.00,10,1149.900,114990.0,ERRO,https://www.amazon.com.br/Display-Interativo-C...,"Display Interativo Cintiq 16, Wacom, D..."
3,B07195NDY9,2022-06-01,AMAZON,Triplenet Pricing INC,458.99,10,45.899,4589.9,ERRO,https://www.amazon.com.br/Folha-Textura-Padr%C...,Folha De Textura Wacom G - Ack122312. ...
4,B0785S55QQ,2022-06-01,AMAZON,BIG BOSS ELETRONICOS,204.90,10,20.490,2049.0,ERRO,https://www.amazon.com.br/Mesa-Digitalizadora-...,Wacom One CTL472 - Mesa Digitalizadora...
...,...,...,...,...,...,...,...,...,...,...,...
213,B07YFG742J,2022-06-01,AMAZON,DONA JUREMA BOUTIK,2135.00,10,213.500,21350.0,ERRO,https://www.amazon.com.br/Digitalizadora-Gaomo...,"Mesa Digitalizadora Gaomon PD1161 11,6..."
214,B07YFG742J,2022-06-01,AMAZON,Promotion.Shop 💎,2185.45,10,218.545,21854.5,ERRO,https://www.amazon.com.br/Digitalizadora-Gaomo...,"Mesa Digitalizadora Gaomon PD1161 11,6..."
215,B084DVTSC9,2022-06-01,AMAZON,Triplenet Pricing INC,402.14,10,40.214,4021.4,ERRO,https://www.amazon.com.br/Wacom-CP91300B2Z-par...,Wacom Uma caneta CP91300B2Z para Wacom...
216,B084DVTSC9,2022-06-01,AMAZON,BUY SUPPLY,811.01,10,81.101,8110.1,ERRO,https://www.amazon.com.br/Wacom-CP91300B2Z-par...,Wacom Uma caneta CP91300B2Z para Wacom...


In [37]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON.xlsx", index=False)
Dataframe_Button.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON_BUTTON.xlsx", index=False)
Dataframe_More.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\AMAZON_MORE.xlsx", index=False)

In [53]:
Df_final = Df_final[Df_final['PRICE'].str.len() < 10]
Df_final = Df_final[Df_final['MORE'] == 0]
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
1,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Black-Pr...,AMAZON,Detona Shop,2269.53,10,226.98,2269.80,B07JH83VBW,Câmera Hero 7 Black à Prova D’água 12M...,Internacional,0
34,2022-05-31,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.60,275.40,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0
35,2022-05-31,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.60,183.60,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0
38,2022-05-31,https://www.amazon.com.br/Zwbfu-acess%C3%B3rio...,AMAZON,Jeneza,177.99,5,35.63,178.15,B09GJMPY6R,Zwbfu Kit de acessórios para câmera de...,Internacional,0
40,2022-05-31,https://www.amazon.com.br/Conjunto-acess%C3%B3...,AMAZON,Minimini,75.00,2,37.50,75.00,B082X8HPLL,Conjunto de acessórios para câmera Mob...,Internacional,0
43,2022-05-31,https://www.amazon.com.br/Kit-50-Acess%C3%B3ri...,AMAZON,Amazon,342.14,10,34.25,342.50,B07LFLJBW4,Kit 50 Em 1 De Acessórios Para Gopro ...,Internacional,0
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.75,208.50,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0
53,2022-05-31,https://www.amazon.com.br/Cucudy-subaqu%C3%A1t...,AMAZON,Jeneza,95.18,3,31.74,95.22,B08TTPW7DW,Cucudy Câmera de ação capa protetora à...,Internacional,0
61,2022-05-31,https://www.amazon.com.br/Zwbfu-XTGP559-prote%...,AMAZON,Jeneza,316.99,10,31.78,317.80,B09GJGW62Z,Zwbfu Câmera XTGP559 Porta de cúpula c...,Internacional,0
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.60,158.00,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0


In [54]:
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
1,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Black-Pr...,AMAZON,Detona Shop,2269.53,10,226.98,2269.80,B07JH83VBW,Câmera Hero 7 Black à Prova D’água 12M...,Internacional,0
34,2022-05-31,https://www.amazon.com.br/Acess%C3%B3rios-para...,AMAZON,FUNPro,275.00,9,30.60,275.40,B086M7FK58,Kit de Acessórios para GoPro Hero 8 Bl...,Internacional,0
35,2022-05-31,https://www.amazon.com.br/fengny-Substitui%C3%...,AMAZON,kokorest,183.40,6,30.60,183.60,B09ZPB2QW3,fengny Substituição de controle remoto...,Internacional,0
38,2022-05-31,https://www.amazon.com.br/Zwbfu-acess%C3%B3rio...,AMAZON,Jeneza,177.99,5,35.63,178.15,B09GJMPY6R,Zwbfu Kit de acessórios para câmera de...,Internacional,0
40,2022-05-31,https://www.amazon.com.br/Conjunto-acess%C3%B3...,AMAZON,Minimini,75.00,2,37.50,75.00,B082X8HPLL,Conjunto de acessórios para câmera Mob...,Internacional,0
43,2022-05-31,https://www.amazon.com.br/Kit-50-Acess%C3%B3ri...,AMAZON,Amazon,342.14,10,34.25,342.50,B07LFLJBW4,Kit 50 Em 1 De Acessórios Para Gopro ...,Internacional,0
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.75,208.50,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0
53,2022-05-31,https://www.amazon.com.br/Cucudy-subaqu%C3%A1t...,AMAZON,Jeneza,95.18,3,31.74,95.22,B08TTPW7DW,Cucudy Câmera de ação capa protetora à...,Internacional,0
61,2022-05-31,https://www.amazon.com.br/Zwbfu-XTGP559-prote%...,AMAZON,Jeneza,316.99,10,31.78,317.80,B09GJGW62Z,Zwbfu Câmera XTGP559 Porta de cúpula c...,Internacional,0
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.60,158.00,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0


In [130]:
url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'])

In [132]:
len(url_names)

21

In [114]:
Df_final.loc[Df_final['ID'] == Dataframe_More['ID'],'URL']

ValueError: Can only compare identically-labeled Series objects

In [57]:
Df_teste = pd.concat([Df_final, Dataframe_Button])
Df_teste = Df_teste.sort_values(by='ID')

In [58]:
Df_teste

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.750,208.50,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0.0
22,2022-05-31,https://www.amazon.com.br/GoPro-Sistema-HERO3-...,AMAZON,LIG Celular,189.9,10,18.990,1899.00,B00J9RO4U2,GoPro Sistema Dual Hero para HERO3+ (c...,ERRO,NaN
14,2022-05-31,https://www.amazon.com.br/GOPOLE-Reach-Mini-ex...,AMAZON,Ami Ventures Inc.,1034.0,10,103.400,10340.00,B01C397F1C,GOPOLE Reach Mini – Haste de extensão ...,ERRO,NaN
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.600,158.00,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0.0
26,2022-05-31,https://www.amazon.com.br/GoPro-Porta-Lateral-...,AMAZON,GRUPO MAVERICK,189.99,10,18.999,1899.90,B01L2CPKOK,"Porta Lateral de Reposição para Hero6,...",ERRO,NaN
20,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,Tech With Expert,4602.0,10,460.200,46020.00,B0792MJLNM,"Câmera Fusion 360, GoPro, Preto",ERRO,NaN
19,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,Porto Seguro Store,3190.0,10,319.000,31900.00,B0792MJLNM,"Câmera Fusion 360, GoPro, Preto",ERRO,NaN
13,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-GoPro-MA...,AMAZON,LIVE IMPORTS,3499.0,10,349.900,34990.00,B07BPVPV14,"Câmera GoPro MAX 360, Preto",ERRO,NaN
12,2022-05-31,https://www.amazon.com.br/C%C3%A2mera-GoPro-MA...,AMAZON,GRUPO MAVERICK,3485.21,10,348.521,34852.10,B07BPVPV14,"Câmera GoPro MAX 360, Preto",ERRO,NaN
11,2022-05-31,https://www.amazon.com.br/Camera-Digital-12-1M...,AMAZON,João e Maria Shop,2574.05,10,257.405,25740.50,B07GSVDFTQ,Camera Digital GoPro Hero 7 Black Ultr...,ERRO,NaN


In [59]:
Df_final_mesmo = pd.concat([Df_teste, Dataframe_More])
Df_final_mesmo

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
44,2022-05-31,https://www.amazon.com.br/GoPole-Alcan%C3%A7a-...,AMAZON,FUNPro,208.40,6,34.750,208.5,B00B3YFDBS,GoPole Alcança de extensão de 14 a 101...,Internacional,0.0
22,2022-05-31,https://www.amazon.com.br/GoPro-Sistema-HERO3-...,AMAZON,LIG Celular,189.9,10,18.990,1899.0,B00J9RO4U2,GoPro Sistema Dual Hero para HERO3+ (c...,ERRO,NaN
14,2022-05-31,https://www.amazon.com.br/GOPOLE-Reach-Mini-ex...,AMAZON,Ami Ventures Inc.,1034.0,10,103.400,10340.0,B01C397F1C,GOPOLE Reach Mini – Haste de extensão ...,ERRO,NaN
62,2022-05-31,https://www.amazon.com.br/transporte-acess%C3%...,AMAZON,wecome2021,158.00,5,31.600,158.0,B01H6OP92G,Estojo de transporte para GoPro Hero(2...,Internacional,0.0
26,2022-05-31,https://www.amazon.com.br/GoPro-Porta-Lateral-...,AMAZON,GRUPO MAVERICK,189.99,10,18.999,1899.9,B01L2CPKOK,"Porta Lateral de Reposição para Hero6,...",ERRO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
144,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,BalbinoShop,3899.99,10,389.999,38999.9,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
145,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,GearNPC,5000.0,10,500.000,50000.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
146,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,eFacini,5072.9,10,507.290,50729.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN
147,2022-05-31,https://www.amazon.com.br/Dji-Osmo-Pocket-2-Co...,AMAZON,Lucas Lapa PhotoPro,5999.0,10,599.900,59990.0,B08HWCCS6S,Dji Osmo Pocket 2 Combo,ERRO,NaN


In [60]:
Df_final_mesmo.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\AMAZON_FINAL_GOPRO.xlsx", index=False)

In [105]:
Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()
#Dataframe_More['URL'] = [Df_final.loc[Df_final['ID'] == Dataframe_More['ID'], 'URL']]

ValueError: Can only compare identically-labeled Series objects

In [117]:
Dataframe_More

,ID,Seller,Price,DATE,URL
0,B07WSRXMS8,LIVE IMPORTS,"R$ 2.299,00",2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...
1,B07WSRXMS8,King Pro St,"R$ 2.289,00",2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...
2,B07WSRXMS8,GRUPO MAVERICK,"R$ 2.599,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...
3,B09D15SFMQ,LIVE IMPORTS,"R$ 3.207,25",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...
4,B09D15SFMQ,King Pro St,"R$ 3.219,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...
5,B09D15SFMQ,GRUPO MAVERICK,"R$ 4.299,99",2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...
6,B09D15SFMQ,MeuDome,"R$ 4.799,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...
7,B08R2XCM9K,EOS IMPORT,"R$ 2.999,00",2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...
8,B08KXN6BNH,King Pro St,"R$ 2.599,00",2022-05-21,NaN
9,B08KXN6BNH,King Pro St,"R$ 2.739,00",2022-05-21,NaN


In [63]:
#Df_final = Df_final.drop(columns=['MORE'])
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL
0,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.90,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional
1,2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...,AMAZON,EOS IMPORT,3207.25,10,320.77,3207.7,B09D15SFMQ,Câmera GoPro HERO10 Black à Prova com ...,Internacional
2,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...,AMAZON,Amazon,4688.20,10,468.82,4688.2,B093JKGDJZ,Câmera GoPro HERO9 Black - Kit Trip Pr...,Internacional
3,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,EOS IMPORT,3000.00,10,300.00,3000.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
4,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,AMAZON,EOS IMPORT,2540.73,10,254.10,2541.0,B08KXN6BNH,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
5,2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...,AMAZON,EOS IMPORT,2599.00,10,259.90,2599.0,B08ZSVP9T7,CÂMERA GOPRO HERO 8 BLACK ESSENCIAL BU...,Internacional
6,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,EOS IMPORT,3799.00,10,379.90,3799.0,B087ZP2GW4,Câmera Digital GoPro Max 360o - CHDHZ-...,Internacional
7,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.29,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional


In [39]:
Df_teste.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Motorola_Amazon.xlsx", index=False)

In [33]:
Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]
print(len(Amazon_seller_more))

187


In [39]:
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]

print(len(Amazon_seller_more))

0


In [37]:
Amazon_seller_more

[' Tudo à Beça',
 'Política de devolução',
 ' Gigantec',
 'Política de devolução',
 ' INFOSHOP.',
 'Política de devolução',
 ' Thaun Shop',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 '',
 'Apagar tudo',
 ' O MUNDO PARA VOCÊ',
 'Política de devolução',
 ' ZAPATA STORE SP',
 'Política de devolução',
 ' OFICINA DOS BITS',
 'Política de devolução',
 ' Syma Solutions',
 'Política de devolução',
 ' Conect Magazine',
 'Política de devolução',
 ' IBYTE',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' MAGIC WORLD ELETRONICOS',
 'Política de devolução',
 ' REX INFORMATICA',
 'Política de devolução',
 ' Tudo à Beça',
 'Política de devolução',
 ' W-TECH IMPORTS',
 'Política de devolução',
 ' Kadri Tecnologia',
 'Política de devolução',
 ' E-SPOT',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 '',
 'Apagar tudo',
 '

In [33]:
Amazon_ID_More = []

In [ ]:
Amazon_seller_more = []
Amazon_price_more = []


In [ ]:
Amazon_seller_more

In [ ]:
Amazon_price_more

In [ ]:

new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B09D15SFMQ" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
    print(seller.text)

for price in Soup.find_all(class_='a-offscreen')[2:]:
    print(price.text)


In [ ]:
for url in Df_more_offers['ID']:
    time.sleep(3)

    search = "https://www.amazon.com.br/gp/product/ajax/?asin=" + url + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(search)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    print(search)

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal'):
        print(seller.text)

In [ ]:
#Teste gráfico 
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(x=Df_final['PRICE'], vert=False)
plt.show()

In [ ]:
df_products = Df_final[Df_final['PRICE'] > 800]

In [ ]:
df_products